# AAO GALAH RESEARCH JAN-APR 2016

### Missy McIntosh, Gayandhi de Silva, Jeffrey Simpson
http://www.univie.ac.at/webda/cgi-bin/frame_data_list.cgi?ascc111+ad2k+ad2000.coo

#### Check if there are any known open clusters in GALAH
#### This is a very slow python script....

In [4]:
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import FK4
import time
from tqdm import *
import pandas as pd
import codecs
import numpy as np
from lxml import html
from lxml import etree
import requests
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import FK4
import csv

/Users/mmcintosh/anaconda/lib/python3.4/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [43]:
def coordconvert(ra, dec):
    c = SkyCoord(ra, dec, unit=(u.hourangle, u.deg))
    newra = c.ra.degree
    newdec = c.dec.degree
    return(newra, newdec)

# grab each cluster coordinate file
## replace var in http://www.univie.ac.at/webda/cgi-bin/frame_data_list.cgi?var+ad2k+ad2000.coo
## go to this link, scrape off RA and DEC data
def getwebda(name):
    linkpt1 = "http://www.univie.ac.at/webda/cgi-bin/frame_data_list.cgi?"
    linkpt2 = "+ad2k+ad2000.coo"
    link = linkpt1+name+linkpt2
    print(name)
    print(link)
    page = requests.get(link)
    tree = html.fromstring(page.content)
    text = str(etree.tostring(tree))
    temp = text.split(r'\n\n ')
    temp = temp[-1]
    temp = temp.split(r'\n')
    temp = temp[:-2]
    temp = [x.split(r'  ') for x in temp]
    coords = [coordconvert(coord[2], coord[3]) for coord in temp]
    subsetra = np.asarray([coord[0] for coord in coords])
    subsetdec = np.asarray([coord[1] for coord in coords])
    return(temp[0], subsetra, subsetdec)

def pbsearch(sorted_data, subset, tolerance):
    tol = tolerance
    searched_arr = []
    matched_arr = []
    
    for i in tqdm(range(len(subset))):
        a = subset[i]
        ra = sorted_data
        n = len(ra)
        time.sleep(3)
        for element in np.arange(0,n):
            index = round(n/2)
            entry = ra[index]
            
            if a - tol > entry:
                ra = ra[index:]
            elif a + tol < entry:
                ra = ra[:(index-1)]
            else:
                matched_arr.append(entry)
                searched_arr.append(a)
                ra = np.delete(ra, index)
            n = len(ra)
    
            if n > 1:
                continue
            else:
                break 
    print("searched for a subset of ", len(subset), "RAs against 2mass list of", len(sorted_data), "RAs")
    print("RA matches: ", len(matched_arr), "with a tolerance of ", tol)
    return(searched_arr,matched_arr)

def checkdec(dec1, dec2, tol):
    if np.absolute((dec1) - (dec2)) < tol:
        return(1)
    else:
        return(0)


In [ ]:
# Read in Gayandhi's 2Mass input database Name, RA, and DEC information
# note this data is presorted by RA
'''

df = pd.read_csv('gsortedgal.csv', delimiter=',',index_col=False, header=0)
df.columns = ["ID", "RA", "DEC"]

# get rid of blank entries
print(len(df))
df = df.loc[(df != 0).any(1)]
print(len(df))
print(df.head())

# assign to np arrays
twomass_ident = np.asarray(df["ID"])
twomass_ra = np.asarray(df["RA"])
twomass_dec = np.asarray(df["DEC"])
'''

In [38]:
# Read in Jeffrey's 2Mass input database Name, RA, and DEC information
# note this data is presorted by RA

df = pd.read_csv('jsortedgal.csv', delimiter=',',index_col=False, header=0, dtype=float)
df.columns = ["ID", "VMag", "RA", "DEC"]

#df.sort_values("RA")
#all(l[i] <= l[i+1] for i in np.arange(len(l)-1))
# there's some weird sorting problems. A few are multiplied by 10^-4 for some reason
l = (df["RA"])
problems = []
for i in np.arange(len(l)-1):
    if (l[i] <= l[i+1]):
        continue
    else:
        problems.append(i+1)
print(len(problems), "weird sorts")
for problem in problems:
    df["RA"][problem] = df["RA"][problem]*10000



print(len(df))
print(df.head)
twomass_ident = np.asarray(df["ID"])
twomass_vmag = np.asarray(df["VMag"])
twomass_ra = np.asarray(df["RA"])
twomass_dec = np.asarray(df["DEC"])

10 weird sorts
9073309
<bound method DataFrame.head of               ID    VMag          RA        DEC
0        1093660  13.429    0.001009 -57.713554
1         179614  12.412    0.001045 -75.687950
2        2969783  11.446    0.001108 -34.697872
3        4979866  12.242    0.001129  -9.856497
4        8247442  12.909    0.001160   3.020907
5        4557755  12.994    0.001162 -15.580398
6        2080468  13.722    0.001469 -45.096962
7        7363787  14.091    0.001501 -37.677406
8        7186563  14.103    0.001561 -44.879086
9         123316  13.274    0.001595 -77.940849
10       1551443  12.032    0.001693 -51.243954
11       8004695  14.040    0.001819  -9.910997
12       7283228  14.124    0.001961 -40.999020
13       6587378  14.234    0.001998 -84.804314
14       1358368  12.960    0.002023 -53.789234
15       3049618  12.999    0.002469 -33.759903
16       6230060  12.743    0.002529   7.491479
17       6152746  12.954    0.002630   6.502540
18       7980033  14.089    0.002

In [ ]:
# Read in DIAS 
# I didn't know an easy way to read in this file so this is very messy and hacked and slow and I apologize
''' 

dias_data = np.genfromtxt('dias.txt', delimiter='\t', names=None, dtype=None)
dias_data = dias_data[1:]

data = []
subsetid = []
subsetra = []
subsetdec = []
for line in dias_data:
 
    line = line.decode('UTF-8')

    line = (str(line).split("  "))

    line = np.array(list(filter(None, line)))
    subsetid.append(line[0])
    try:
        c = SkyCoord(line[1], line[2], unit=(u.hourangle, u.deg))
        subsetra.append(c.ra.degree)
        subsetdec.append(c.dec.degree)
    except:
        print("error: ", line)
    data.append(line)
    
# some problems with dec
subsetra = np.asarray(subsetra)
subsetdec = np.asarray(subsetdec)
name = "dias"
'''

In [40]:
# Read in WEBDA

#I got this html from querying webda for all clusters with J2000 coordinate data available

# connect to webda database
# grab each cluster name
## search this page: http://www.univie.ac.at/webda/cgi-bin/seldb.cgi?
## for links like http://www.univie.ac.at/webda/cgi-bin/ocl_page.cgi?dirname=wes02
## and grab the string after dirname=  


f = codecs.open("clustershtml.html", "r", "utf-8")
page = f.read()
split = page.split("dirname=")
names = [split[x].split('"')[0] for x in np.arange(len(split))]
names = names[1:]
print(len(names))
i = 1
name = names[i]
subsetid, subsetra, subsetdec = getwebda(name)

946
ale02
http://www.univie.ac.at/webda/cgi-bin/frame_data_list.cgi?ale02+ad2k+ad2000.coo


In [41]:
print(twomass_ra[1000:1005])
print(twomass_dec[1000:1005])

# add in fake positive matches at the very end to make sure the code works
subsetra = np.append(subsetra, twomass_ra[1000:1005])
subsetdec = np.append(subsetdec, twomass_dec[1000:1005])



[ 0.118115  0.118397  0.118726  0.119066  0.119151]
[  0.736194   3.603777 -32.138985 -64.530884 -49.611828]


In [ ]:
# search each coord in file against 2 mass

# return subset RAs found within sorted data
# tolernece is 20 arc seconds
tol = 0.005556
searched_ra, matched_ra = pbsearch(twomass_ra, subsetra, tol)



 37%|███▋      | 17/46 [00:51<01:27,  3.03s/it]

In [35]:
l = (df["RA"])
problems = []
for i in np.arange(len(l)-1):
    if (l[i] <= l[i+1]):
        continue
    else:
        problems.append(i)
        print(i)

19611
26398
33178
39119
39213
69043
70513
71098
74894
80128


In [37]:

print(len(problems), "weird sorts")
for problem in problems:
    df["RA"][problem+1] = df["RA"][problem+1]*10000

problem = 80128
temp1 = twomass_ra[problem]
temp2 = twomass_ra[problem+1]*10000
temp3 = twomass_ra[problem+2]
#(temp[0] <= temp[1])
print(temp1, temp2, temp3)

10 weird sorts
96092830.0 9.61 9.610174


In [42]:
temp = twomass_ra[753120:753125]
print(temp)
n = 3
index = np.array(temp[n])
print(type(index))
print(index)
num = np.delete(temp, n)
print(num)
arr = np.concatenate(temp[:(n-1)], temp[(n+1):])
print(arr)

test = [1,2,3,4,5,6]
np.delete(test, 4)

[ 71.204034  71.204218  71.204377  71.204438  71.204616]
<class 'numpy.ndarray'>
71.204438
[ 71.204034  71.204218  71.204377  71.204616]
[ 71.204034  71.204218]


/Users/mmcintosh/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:9: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


array([1, 2, 3, 4, 6])

In [ ]:
# retrieve the DEC and indicies of the found subset RAs
# check that the DEC values are w/i tolerence and return matched indices
# indices returned are for 2MASS

searcheddec_index = [np.where(subsetra == ra)[0][0] for ra in searched_ra] 
searcheddec = [subsetdec[x] for x in searcheddec_index]

matcheddec_index = [np.where(twomass_ra == ra)[0][0] for ra in matched_ra]  
matcheddec = [twomass_dec[x] for x in matcheddec_index]



In [ ]:
bools = []
for i in np.arange(len(matcheddec_index)):
    bol = checkdec(matcheddec[i],searcheddec[i], tol)
    bools.append(bol)


# select info on the matched RA and DECs
n= np.sum(bools)
if n > 0:
    print(n, "matches for", name, "for tolerance", tol)
    fullmatchindex = np.where(bools)[0]
    fullmatchindex = [int(x) for x in fullmatchindex]
    subsetids = [subsetid[index] for index in fullmatchindex]
    galahids = [twomass_ident[index] for index in fullmatchindex]
    filename = name+"_wtol"+str(tol)+"_matched"+str(n)+'.csv'
    with open(filename, 'w', newline='') as fp:
        a = csv.writer(fp, delimiter=',')
        data = [subsetids, galahids]
        a.writerows(data)
else:
    print("No matches for", name, "for tolerance", tol)
   

In [ ]:
#debug

print(len(twomass_ident))
print(len(subsetra))


print(twomass_ra[1000:1005])
print(twomass_dec[1000:1005])

# add in fake positive matches at the very end to make sure the code works
subsetra = np.append(subsetra, twomass_ra[1000:1005])
subsetdec = np.append(subsetdec, twomass_dec[1000:1005])



In [ ]:
# back up for webda, download files individually
# Read in Webda observed Name, RA, and DEC information

df = pd.read_csv('ad2000.coo', delimiter='\t',index_col=False, header=1)
df.columns = ["ID", "ref", "RA", "DEC"]


coords = [coordconvert(ra, dec) for ra in df["RA"] for dec in df["DEC"]]
subsetra = np.asarray([coord[0] for coord in coords])
subsetdec = np.asarray([coord[1] for coord in coords])